In [ ]:
%%time
import numpy as np

N,D_in,H,D_out = 64,1000,100,10 
x= np.random.randn(N, D_in)
y=np.random.randn(N,D_out)
w1= np.random.randn(D_in,H)
w2=np.random.randn(H,D_out)
LR=1e-6


for t in range(400):
    # 순전파 단계 
    h=x.dot(w1)
    h_relu=np.maximum(h,0)
    y_pred=h_relu.dot(w2)

    # loss 를 계산하고 출력한다.
    loss= np.square(y_pred-y).sum()
    print(t, loss)

    #손실에 따른 w1,w2의 변화도를 계산하고 역전파합니다.
    grad_y_pred=2.0*(y_pred-y)
    grad_w2=h_relu.T.dot(grad_y_pred)
    grad_h_relu=grad_y_pred.dot(w2.T)
    grad_h=grad_h_relu.copy()
    grad_h[h<0]=0
    grad_w1=x.T.dot(grad_h)

    #가중치를 갱신합니다 
    w1-=LR*grad_w1
    w2-=LR*grad_w2




In [ ]:
import torch 
dtype=torch.float
device=torch.device('cuda' if torch.cuda.is_available else 'cpu')
print(device)
N,D_in, H,D_out =64,1000,100,10
x=torch.randn(N,D_in,device=device,dtype=dtype)
y=torch.randn(N,D_out,device=device,dtype=dtype)
# 무작위로 가중치를 초기화하기
w1=torch.randn(D_in,H,device=device,dtype=dtype)
w2=torch.randn(H,D_out,device=device,dtype=dtype)
LR=1e-5/11
print(x.shape,w1.shape,y.shape,w2.shape)
for t in range(500):
    h=x.matmul(w1)
    h_relu=h.clamp(min=0)
    y_pred=h_relu.matmul(w2)

    # loss 를 계산하고 출력한다
    loss=(y_pred-y).pow(2).sum().item()
    if t%10==0:
        print(t,loss)
    # 손실에 따른 w1,w2의 변화도를 계산하고 역전파합니다
    grad_y_pred=2.0*(y_pred-y)
    grad_w2=h_relu.t().matmul(grad_y_pred)
    grad_h_relu=grad_y_pred.matmul(w2.t())
    grad_h=grad_h_relu.clone()
    grad_h[h<0]=0
    grad_w1=x.t().matmul(grad_h)

    # 경사하강법을 사용하여 가중치를 갱신한다.
    w1-=LR*grad_w1
    w2-=LR*grad_w2

In [ ]:
import torch 
dtype=torch.float
device=torch.device('cuda' if torch.cuda.is_available else 'cpu')
print(device)
N,D_in, H,D_out =64,1000,100,10
x=torch.randn(N,D_in,device=device,dtype=dtype)
y=torch.randn(N,D_out,device=device,dtype=dtype)
# 무작위로 가중치를 초기화하기
w1=torch.randn(D_in,H,device=device,dtype=dtype,requires_grad=True)
w2=torch.randn(H,D_out,device=device,dtype=dtype,requires_grad=True)
LR=1e-5/11
print(x.shape,w1.shape,y.shape,w2.shape)
for t in range(500):
    # h=x.matmul(w1)
    # h_relu=h.clamp(min=0)
    # y_pred=h_relu.matmul(w2)
    # 중간단계들에 reference를 가지고 있을 필요가없으므로 한번에 구한다
    y_pred=x.mm(w1).clamp(min=0).mm(w2)
    # loss 를 계산하고 출력한다
    loss=(y_pred-y).pow(2).sum()
    if t%10==0:
        print(t,loss.item())
    # 손실에 따른 w1,w2의 변화도를 계산하고 역전파합니다
    # grad_y_pred=2.0*(y_pred-y)
    # grad_w2=h_relu.t().matmul(grad_y_pred)
    # grad_h_relu=grad_y_pred.matmul(w2.t())
    # grad_h=grad_h_relu.clone()
    # grad_h[h<0]=0
    # grad_w1=x.t().matmul(grad_h)
    loss.backward()
    # 경사하강법을 사용하여 가중치를 갱신한다.
    with torch.no_grad():
        w1-=LR*w1.grad
        w2-=LR*w2.grad
        #가중치 갱신후에는 수동으로 변화도를 0으로 만들어준다.
        w1.grad.zero_()
        w2.grad.zero_()

In [ ]:
# -*- coding: utf-8 -*-
import torch

# N은 배치 크기이며, D_in은 입력의 차원입니다;
# H는 은닉층의 차원이며, D_out은 출력 차원입니다.
N, D_in, H, D_out = 64, 1000, 100, 10

# 입력과 출력을 저장하기 위해 무작위 값을 갖는 Tensor를 생성합니다.
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# nn 패키지를 사용하여 모델을 순차적 계층(sequence of layers)으로 정의합니다.
# nn.Sequential은 다른 Module들을 포함하는 Module로, 그 Module들을 순차적으로
# 적용하여 출력을 생성합니다. 각각의 Linear Module은 선형 함수를 사용하여
# 입력으로부터 출력을 계산하고, 내부 Tensor에 가중치와 편향을 저장합니다.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

# 또한 nn 패키지에는 널리 사용하는 손실 함수들에 대한 정의도 포함하고 있습니다;
# 여기에서는 평균 제곱 오차(MSE; Mean Squared Error)를 손실 함수로 사용하겠습니다.
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-4
for t in range(500):
    # 순전파 단계: 모델에 x를 전달하여 예상되는 y 값을 계산합니다. Module 객체는
    # __call__ 연산자를 덮어써(override) 함수처럼 호출할 수 있게 합니다.
    # 이렇게 함으로써 입력 데이터의 Tensor를 Module에 전달하여 출력 데이터의
    # Tensor를 생성합니다.
    y_pred = model(x)

    # 손실을 계산하고 출력합니다. 예측한 y와 정답인 y를 갖는 Tensor들을 전달하고,
    # 손실 함수는 손실 값을 갖는 Tensor를 반환합니다.
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # 역전파 단계를 실행하기 전에 변화도를 0으로 만듭니다.

    # 역전파 단계: 모델의 학습 가능한 모든 매개변수에 대해 손실의 변화도를
    # 계산합니다. 내부적으로 각 Module의 매개변수는 requires_grad=True 일 때
    # Tensor 내에 저장되므로, 이 호출은 모든 모델의 모든 학습 가능한 매개변수의
    # 변화도를 계산하게 됩니다.
    loss.backward()

    # 경사하강법(gradient descent)를 사용하여 가중치를 갱신합니다. 각 매개변수는
    # Tensor이므로 이전에 했던 것과 같이 변화도에 접근할 수 있습니다.
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad
    model.zero_grad()

In [ ]:
# -*- coding: utf-8 -*-
import torch

# N은 배치 크기이며, D_in은 입력의 차원입니다;
# H는 은닉층의 차원이며, D_out은 출력 차원입니다.
N, D_in, H, D_out = 64, 1000, 100, 10

# 입력과 출력을 저장하기 위해 무작위 값을 갖는 Tensor를 생성합니다.
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# nn 패키지를 사용하여 모델과 손실 함수를 정의합니다.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)
loss_fn = torch.nn.MSELoss(reduction='sum')

# optim 패키지를 사용하여 모델의 가중치를 갱신할 Optimizer를 정의합니다.
# 여기서는 Adam을 사용하겠습니다; optim 패키지는 다른 다양한 최적화 알고리즘을
# 포함하고 있습니다. Adam 생성자의 첫번째 인자는 어떤 Tensor가 갱신되어야 하는지
# 알려줍니다.
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for t in range(500):
    # 순전파 단계: 모델에 x를 전달하여 예상되는 y 값을 계산합니다.
    y_pred = model(x)

    # 손실을 계산하고 출력합니다.
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # 역전파 단계 전에, Optimizer 객체를 사용하여 (모델의 학습 가능한 가중치인)
    # 갱신할 변수들에 대한 모든 변화도를 0으로 만듭니다. 이렇게 하는 이유는
    # 기본적으로 .backward()를 호출할 때마다 변화도가 버퍼(buffer)에 (덮어쓰지 않고)
    # 누적되기 때문입니다. 더 자세한 내용은 torch.autograd.backward에 대한 문서를
    # 참조하세요.
    optimizer.zero_grad()

    # 역전파 단계: 모델의 매개변수에 대한 손실의 변화도를 계산합니다.
    loss.backward()

    # Optimizer의 step 함수를 호출하면 매개변수가 갱신됩니다.
    optimizer.step()

In [ ]:
class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in,H,D_out):
        super(TwoLayerNet,self).__init__()
        self.linear1=torch.nn.Linear(D_in,H)
        self.linear2=torch.nn.Linear(H,D_out)
    
    def forward(self,x):
        '''순전파 함수에서는 입력 데이터의 tensor를 받고 출력 데이터의 tensor를 반환해야 한다.
        Tensor상의 임의의 연산자뿐만 아니라 생성자에서 정의한 Module도 사용할 수 있다.'''
        h_relu = self.linear1(x).clamp(min=0)
        y_pred=self.linear2(h_relu)
        return y_pred

N, D_in, H, D_out = 64, 1000, 100, 10
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)
model = TwoLayerNet(D_in, H, D_out)
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
for t in range(500):
    y_pred = model(x)
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

### pytorch 
- 제어 흐름(Contorl Flow)+가중치공유(weight sharing)
```
동적 그래프와 가중치 공유의 예로, 매우 이상한 모델을 구현해보겟다.각 순전파 단계에서 많은 은닉 계층을 갖는 완전히 연결(fully connected)된 Relu신경망이 무자구이로 0~3 사이의 숫자를 선택하고, 가장 안쪽(innerMost)의 
은닉층들을 계산하기 위해 동일한 가중치를 여러 번 재사용합니다 

이 모델에서는 일반적인 Python 제어 흐름을 사용하여 반복을 구현할 수 있으며, 순전파 단계를 정의할 때 단지 동일한 module 을 여러번 재사용함으로써 내부계층들 간의 가중치 공유를 구현할 수 있습니다.
이러한 모델을 Moduel을 상속받는 서브클래스로 간단히 구현해보겟습니다.
```

In [ ]:
import random
import pandas as pd
class DynamicNet(torch.nn.Module):
    def __init__(self, D_in,H,D_out):
        super(DynamicNet,self).__init__()
        self.input_linear=torch.nn.Linear(D_in,H)
        self.middle_linear=torch.nn.Linear(H,H)
        self.output_linear=torch.nn.Linear(H,D_out)

    def forward(self, x):
        h_relu=self.input_linear(x).clamp(min=0)
        for _ in range(random.randint(0,3)):
            h_relu=self.middle_linear(h_relu).clamp(min=0)
        y_pred=self.output_linear(h_relu)
        return y_pred


N,D_in,H,D_out=64,1000,100,10

x=torch.randn(N,D_in)
y=torch.randn(N,D_out)

medel=DynamicNet(D_in,H,D_out)

# 손실함수와 옵티마이저 생성 
# -> 이 요상한 모델은 SGD로 구할 수 없으니 momentum을 이용한다고 한다.
criterion=torch.nn.MSELoss(reduction='sum')
optimizer=torch.optim.SGD(model.parameters(),lr=1e-4,momentum=0.9)

for t in range(500):
    y_pred=model(x)
    loss=criterion(y_pred,y)
    if t%100==0:
        print(t,loss.item())
    # 변화도를 0으로 만들고 역전파 단계를 수행하고 가중치를 갱신한다.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    

### MNiST 데이터 준비 


In [ ]:
from pathlib import Path
import requests

Data_path=Path('data')
PATH=Data_path/'mnist'
PATH.mkdir(parents=True,exist_ok=True)

URL='https://deeplearning.net/data/mnist/'
FileName='mnist.pkl.gz'

if not(PATH/FileName).exists():
    content=requests.get(URL+FileName).content
    (PATH/FileName).open('wb').write(content)


In [ ]:
import tensorflow as tf
data_train,data_test=tf.keras.datasets.mnist.load_data()
image_train,label_train=data_train
image_test,label_test=data_test

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

plt.imshow(image_train[2].reshape((28,28)),cmap='gray')
print(x_train.shape)

In [ ]:
import torch
x_train,y_train=data_train
x_vaild,y_vaild=data_test
x_train,y_train,x_vaild,y_vaild=map(torch.tensor,(x_train,y_train,x_vaild,y_vaild))
print( x_train.shape)
x_train,x_train.shape,y_train.min(),y_train.max()
print(x_train,y_train)
print(x_train.reshape(-1,28*28).shape)
print(y_train.min(),y_train.max())

In [ ]:
import math
w1= torch.randn(784,10)/math.sqrt(784)
w1.requires_grad_()
bias=torch.zeros(10,requires_grad=True)
w1

In [ ]:
def log_softmax(x):
    return x-x.exp().sum(-1).log().unsqueeze(-1)

def model(xb):
    return log_softmax(xb.mm(w1)+bias)

In [ ]:
bs=64
xb=x_train.reshape(-1,(28*28))[0:bs]
print(xb.shape)
preds=model(xb)

preds.shape

###TensorBoard 로 모델 , 데이터 , 학습 시각화하기
---

1. 데이터를 읽고 적적히 변화합니다.
2. TensorBoard를 set up 합니다.
3. TensorBoard에 write 합니다
4. TensorBoard를 사용하여 모델 구조를 살펴봅니다
5. 약가의 코드를 추가하여 TensorBoard에서 이전 듀토리얼에서 만든 시각화의 대화식 버전을 만듭니다
---
- 학습데이터를 검사하는 몇가지 방법
- 학습에 따른 모델의 성능을 추적하는 방법
- 학습이 완료된 모델의 성능 평가하는 방법


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch 
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim 
# transform 
transform =transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,),(0.5,))])
#datasets
trainset=torchvision.datasets.FashionMNIST('./data',download=True,train=True,transform=transform)
testset=torchvision.datasets.FashionMNIST('./data',download=True,train=False,transform=transform)
#dataloaders
trainloader=torch.utils.data.DataLoader(trainset,batch_size=4,shuffle=True,num_workers=2)
testloader=torch.utils.data.DataLoader(testset,batch_size=4,shuffle=False,num_workers=2)

#분류 결과를 위한 상수 
classes=('T-shirt/top','Trouse','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle Bot')
#이미지를 보여주기 위한 헬퍼 함수
def matplotlib_imshow(img,one_channel=False):
    if one_channel:
        img=img.mean(dim=0)
    img=img/2+0.5
    npimg=img.numpy()
    if one_channel:
        plt.imshow(npimg,cmap='Greys')
    else:
        plt.imshow(np.transpose(npimg,(1,2,0)))


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1=nn.Conv2d(1,6,5)
        self.pool=nn.MaxPool2d(2,2)
        self.conv2=nn.Conv2d(6,16,5)
        self.fc1=nn.Linear(16*4*4,120)
        self.fc2=nn.Linear(120,84)
        self.fc3=nn.Linear(84,10)

    def forward(self,x):
        x=self.pool(F.relu(self.conv1(x)))
        x=self.pool(F.relu(self.conv2(x)))
        x=x.view(-1,16*4*4)
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x

net=Net()
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(net.parameters(),lr=0.001)

In [ ]:
# TensorBoard 설정

from torch.utils.tensorboard import SummaryWriter
# 기본 log_dir 은 runs이며 여기서는 더 구체적으로 지정하였다. 
writer=SummaryWriter('runs/fashion_mnist_experiment_1')

# TensorBoard 에 기록하기 

dataiter=iter(trainloader)
images,labels=dataiter.next()
#이미지 그리드를 만듭니다 
img_grid=torchvision.utils.make_grid(images)

# 이미지를 보여줍니다.
matplotlib_imshow(img_grid,one_channel=True)

# 텐서보드에 기록합니다 
writer.add_image('four_fashion_mnist_images',img_grid)


In [ ]:
writer.add_graph(net,images)
writer.close()

In [ ]:
def select_n_random(data, labels, n=100):
    '''
    데이터셋에서 n개의 임의의 데이터포인트(datapoint)와 그에 해당하는 라벨을 선택합니다
    '''
    assert len(data) == len(labels)

    perm = torch.randperm(len(data))
    return data[perm][:n], labels[perm][:n]

# 임의의 이미지들과 정답(target) 인덱스를 선택합니다
images, labels = select_n_random(trainset.data, trainset.targets)

# 각 이미지의 분류 라벨(class label)을 가져옵니다
class_labels = [classes[lab] for lab in labels]

# 임베딩(embedding) 내역을 기록합니다
features = images.view(-1, 28 * 28)
writer.add_embedding(features,
                    metadata=class_labels,
                    label_img=images.unsqueeze(1))
writer.close()

### TEXT 
---
- 모델 정의하기 

In [26]:
import math
import torch 
import torch.nn as nn
import torch.nn.functional as F 

class TransformerModel(nn.Module):

    def __init__(self, ntoken, ninp, nhead,nhid, nlayers,dropout=0.5):
        super(TransformerModel,self).__init__()
        from torch.nn import TransformerEncoder,TransformerEncoderLayer
        self.model_type='Transformer'
        self.src_mask=None
        self.pos_encoder=PositionalEncoding(ninp,dropout)
        encoder_layers=TransformerEncoderLayer(ninp,nhead,nhid,dropout)
        self.transformer_encoder=TransformerEncoder(encoder_layers,nlayers)
        self.encoder=nn.Embedding(ntoken,ninp)
        self.ninp=ninp
        self.decoder=nn.Linear(ninp,ntoken)
        self.init_weights()

    def _generate_square_subsequent_mask(self,sz):
        mask=(torch.triu(torch.ones(sz,sz))==1).transpose(0,1)
        mask=mask.float().masked_fill(mask==0,float('-inf')).masked_fill(mask==1,float(0.0))
        return mask
    
    def init_weights(self):
        initrange=0.1
        self.encoder.weight.data.uniform_(-initrange,initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange,initrange)

    def forward(self,src):
        if self.src_mask is None or self.src_mask.size(0)!=len(src):
            device=src.device
            mask=self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask=mask

            src=self.encoder(src)*math.sqrt(self.ninp)
            src=self.pos_encoder(src)
            output=self.transformer_encoder(src,self.src_mask)
            output=self.decoder(output)
            return output
            

In [27]:
class PositionalEncoding(nn.Module):
    def __init__(self,d_model,dropout=0.1, max_len=5000):
        super(PositionalEncoding,self).__init__()
        self.dropout=nn.Dropout(p=dropout)

        pe=torch.zeros(max_len,d_model)
        position=torch.arange(0,max_len,dtype=torch.float).unsqueeze(1)
        div_term=torch.exp(torch.arange(0,d_model,2).float()*(-math.log(10000.0)/d_model))
        pe[:,0::2]=torch.sin(position*div_term)
        pe[:,1::2]=torch.cos(position*div_term)
        pe=pe.unsqueeze(0).transpose(0,1)
        self.register_buffer('pe',pe)

    def forward(self,x):
        x=x+self.pe[:x.size(0),:]
        return self.dropout(x)



### 데이터 로드하고 배치 만들기
---


In [28]:
import torchtext
from torchtext.data.utils import get_tokenizer
TEXT=torchtext.data.Field(
    tokenize=get_tokenizer('basic_english'),
    init_token='<sos>',
    eos_token='<eos>',
    lower=True
)
train_txt, val_txt,test_txt=torchtext.datasets.WikiText2.splits(TEXT)
TEXT.build_vocab(train_txt)
device=torch.device('cuda' if torch.cuda.is_available else 'cpu')

def batchify(data,bsz):
    data=TEXT.numericalize([data.examples[0].text])
    #데이터셋을 bsz파트들로 나눈다
    nbatch=data.size(0)//bsz
    #깔끔하게 나누어 떨어지지 않는 추가적인 부분은 잘라낸다
    data=data.narrow(0,0,nbatch*bsz)
    #데이터에 대하여 bsz배치들로 동등하게 나눈다
    data=data.view(bsz,-1).t().contiguous()
    return data.to(device)

batch_size=20
eval_batch_size=10
train_data=batchify(train_txt,batch_size)
val_data=batchify(val_txt,eval_batch_size)
test_data=batchify(test_txt,eval_batch_size)
train_data.shape


torch.Size([104335, 20])


### input과 target 시퀀스를 생성하기 위한 함수들
---
```
gat_batch() 함수는 트랜스포머 모델을 위한 입력과 타켓시퀀스를 생성합니다. 이 함수는 소스 데이터를 bptt 
길이를 가진 덩어리로 세분화 합니다. 언어 모델링 과제를 위해서, 모델은 다음 단어인 target이 필요합니다 예를 
들어, bptt 의값이 2라면 우리는 i=0 일떄 다음의 2개의 변수(variable)를 얻을 수 있습니다.
```
|Input ||| Target|
|:------|------:|
|A G M S||| B H N T|
|B H B T||| C I O U|

In [29]:
bptt =35
def get_batch(source,i):
    seq_len=min(bptt,len(source)-1-i)
    data=source[i:i+seq_len]
    target=source[i+1:i+1+seq_len].view(-1)
    return data, target
    

### 인스턴스(instance) 초기화하기


In [30]:
ntokens=len(TEXT.vocab.stoi)# 단어 사전의 크기
emsize=200#임베딩 차원
nhid=200 #nn.TransformerEncoder에서 피드포워드 네트워크 모델의 차원
nlayers= 2 
nhead=2 # 멀티헤드 어텐션 모델의 헤드 개수 
dropout=0.2 # 드랍아웃 값 
model=TransformerModel(ntokens,emsize,nhead,nhid,nlayers,dropout).to(device)


```
PositionalEncoding 모듈은 시퀀스 안에서 토큰의 상대적인 또는 절대적인 포지션에 대한 어떤 정보를

주입합닏. 포지셔널 인코딩은 임베딩과 합칠 수 있도록 같은 차원을 가집니다.

여기에서 우리는 다른 주파수의 sin과 cos함수를 사용합니다 
```

### 모델 실행하기 
---

```
 loss을 추적하는 데에는 `CrossEntropyLoss` 가 적용되며, 옵티마이저로서 SGD는 확률적 경사 하강법을
 구현합니다. 초기 학습률은 5.0으로 설정됩니다.`stepLR`은 에포크에 따라서 학습률을 조절하는데 사용됩니다. 
 학습하는 동안에,우리는 기울기 폭발(gradient Exploding)을 방지하기 위하여 모든 기울기를 함께 스케일(scale)
 하는 함수인`nn.utils.clip_grad_norm_`을 이용합니다
```

In [31]:
criterion = nn.CrossEntropyLoss()
lr = 5.0 # 학습률
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

import time
def train():
    model.train() # 학습 모드를 시작합니다.
    total_loss = 0.
    start_time = time.time()
    ntokens = len(TEXT.vocab.stoi)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        log_interval = 200
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.2f} | ms/batch {:5.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                    epoch, batch, len(train_data) // bptt, scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

def evaluate(eval_model, data_source):
    eval_model.eval() # 평가 모드를 시작합니다.
    total_loss = 0.
    ntokens = len(TEXT.vocab.stoi)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, bptt):
            data, targets = get_batch(data_source, i)
            output = eval_model(data)
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, targets).item()
    return total_loss / (len(data_source) - 1)

In [34]:
best_val_loss = float("inf")
epochs = 3 # 에포크 수
best_model = None
for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train()
    val_loss = evaluate(model, val_data)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
          'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, math.exp(val_loss)))
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model

    scheduler.step()

AttributeError: 'NoneType' object has no attribute 'view'

In [ ]:
test_loss = evaluate(best_model, test_data)
print('=' * 89)
print('| End of training | test loss {:5.2f} | test ppl {:8.2f}'.format(
    test_loss, math.exp(test_loss)))
print('=' * 89)